In [1]:
%%capture

!pip install datasets
!pip install transformers==4.28.0
!pip install accelerate -U
!pip install --upgrade accelerate

import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
import torch

In [2]:
dataset_train = load_dataset('IlyaGusev/gazeta', revision="v1.0", split= 'train[:20%]')
dataset_test = load_dataset('IlyaGusev/gazeta', revision="v1.0", split= 'test[:20%]')

No config specified, defaulting to: gazeta/default
Found cached dataset gazeta (C:/Users/olya/.cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457)
No config specified, defaulting to: gazeta/default
Found cached dataset gazeta (C:/Users/olya/.cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457)


In [3]:
type(dataset_train)

datasets.arrow_dataset.Dataset

In [4]:
type(dataset_test)

datasets.arrow_dataset.Dataset

In [5]:
#Общий вид
dataset_test['text'][0]

'Американское аэрокосмическое агентство NASA огласило названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации и запуск которых может состояться уже в конце этого десятилетия. Эти четыре проекта стали полуфиналистами конкурса, объявленного среди американских научных команд, в котором участвовало более десяти миссий. Все они были отобраны по критериям потенциальной пользы для науки и технической осуществимости проекта. В рамках программы Discovery NASA занимается планированием миссий, которые призваны дать ответы на фундаментальные вопросы о происхождении тел Солнечной системы и возможному наличию жизни на них. «Эти выбранные миссии могут трансформировать наше восприятие некоторых из наиболее активных и сложных миров в Солнечной системе, — заявил Томас Зурбучен, помощник директора NASA по науке. — Исследование каждого из этих небесных тел поможет раскрыть секреты о том, как они и им подобные объекты образовались в космосе». Каждый проект из выбранн

In [6]:
dataset_test['summary'][0]

'В NASA назвали четыре миссии в дальний космос, которые в этом десятилетии могут быть запущены американцами. Среди них — две миссии по изучению Венеры, полет к спутнику Юпитера и экспедиция к Тритону, спутнику Нептуна.'

In [7]:
model_name = "IlyaGusev/rut5_base_sum_gazeta"

In [8]:
max_len_sum, max_len_tl = 80, 20

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    tokenized_input = tokenizer(batch['summary'], padding='max_length', truncation=True, max_length=max_len_sum)
    tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tl)

    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

dataset_train = dataset_train.map(tokenize, batched=True, batch_size=8)
dataset_test = dataset_test.map(tokenize, batched=True, batch_size=8)

dataset_train.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
dataset_test.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

Downloading:   0%|          | 0.00/279 [00:00<?, ?B/s]

C:\Users\olya\anaconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\olya\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/828k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Map:   0%|          | 0/10480 [00:00<?, ? examples/s]

Map:   0%|          | 0/1154 [00:00<?, ? examples/s]

In [10]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/766 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/977M [00:00<?, ?B/s]

In [12]:
from fastai.vision.all import partial
from functools import partial

training_args = TrainingArguments(
    "run_gazeta",
    evaluation_strategy = "epoch",
    learning_rate=0.00001,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    remove_unused_columns=True,
    save_strategy='no',
    report_to='none',
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test
)

trainer.train()

C:\Users\olya\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


In [ ]:
INX = 100
NMB = 10

In [42]:
input_text = dataset_test['summary'][INX: INX+NMB]

device = "cuda"

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=7,
        temperature = 1.3,
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2
    )
    
    for i in range(NMB):
        pred = tokenizer.decode(generated_ids[i], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        print("SUMMARY: | {}".format(dataset_test['summary'][INX+i]))
        print("SUMMARY ПО РЕЗУЛЬТАТАМ НЕЙРОСЕТИ: | {}".format(pred))
        print()

SUMMARY: | Российские фрегаты «Адмирал Макаров» и «Адмирал Григорович» с крылатыми ракетами «Калибр-НК» проходят через турецкие проливы Босфор и Дарданеллы в направлении Средиземного моря. Ранее эти корабли уже участвовали в военной операции РФ в Сирии и теперь направляются туда на фоне обострившейся ситуации в Идлибе.
SUMMARY ПО РЕЗУЛЬТАТАМ НЕЙРОСЕТИ: | Российские фрегаты проходят через Босфор и Дарданеллы

SUMMARY: | Протесты украинских шахтеров вынудили Раду выделить дополнительные $40 млн угледобывающей промышленности. Этих денег хватит на погашение текущих долгов, однако не за горами новые: объемы производства сокращаются, средств не хватает даже для освещения шахт, а уже добытый уголь никому не нужен и лежит на складах.
SUMMARY ПО РЕЗУЛЬТАТАМ НЕЙРОСЕТИ: | Украинские шахтеры тратят на уголь

SUMMARY: | Зарубежные спецслужбы намеренно ищут уязвимости в российском IT-секторе, чтобы проводить масштабные кибератаки, заявил секретарь Совбеза Николай Патрушев. По его словам, основные це

In [59]:
# Ваш новостной текст, это вывод работы модели

text = '''Дефицит федерального бюджета России по итогам апреля превысил план на 2023 год, следует из предварительной оценки Минфина. 
По итогам января — апреля бюджет был исполнен с дефицитом до 3,424 трлн рублей, увеличившись в апреле сразу на 1,024 трлн рублей. 
Таким образом, дефицит бюджета уже на 843 млрд рублей превосходит расчеты Минфина, 
который ожидал, что в 2023 году дефицит составит 2,925 трлн рублей, или 2% ВВП'''

input_text = text
device = "cuda"

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=7,
        temperature = 1.3,
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2
    )
    
    for i in range(NMB):
        pred = tokenizer.decode(generated_ids[i], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        print("ВАШ ТЕКСТ: | {}".format(text))
        print("SUMMARY ПО РЕЗУЛЬТАТАМ НЕЙРОСЕТИ: | {}".format(pred))
        print()

#IndexError-не обращать внимание, эта ошибка будет не всегда появляться, она ни на что не влияет, саммари НС для рандомного текста делает

ВАШ ТЕКСТ: | Дефицит федерального бюджета России по итогам апреля превысил план на 2023 год, следует из предварительной оценки Минфина. 
По итогам января — апреля бюджет был исполнен с дефицитом до 3,424 трлн рублей, увеличившись в апреле сразу на 1,024 трлн рублей. 
Таким образом, дефицит бюджета уже на 843 млрд рублей превосходит расчеты Минфина, 
который ожидал, что в 2023 году дефицит составит 2,925 трлн рублей, или 2% ВВП
SUMMARY ПО РЕЗУЛЬТАТАМ НЕЙРОСЕТИ: | Дефицит превысил план на 2023 год



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 11>:30                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: index 1 is out of bounds for dimension 0 with size 1